In [1]:
import os
os.getcwd()

'C:\\Users\\AdminUser'

In [2]:
import matplotlib
import pandas as pd
import numpy as np
import json
import math
import urllib.request
import dateutil.parser
import dateutil.rrule
import datetime
import pytz
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import collections
from textwrap import wrap

In [3]:
matplotlib.rcParams.update({
    'font.size':13,
    'timezone': 'Europe/London'
})

In [4]:
# Used across all of the plots 
dateToday = datetime.datetime.combine(datetime.date.today(),datetime.datetime.min.time())

# How many seconds between readings in trhe files that are generated. 900s = 15 mins data
resampleFrequency = 900

In [5]:
# These timestamps are in ISO8601 format. You can Google this for more information.
# YYYY-MM-DDTHH:MM:SSZ - always in UTC, no British Summer Time
# Be aware that you probably need to request a little bit more than you want, because
# otherwise there could be gaps at the start and end where the value didn't change...
requestFrom = '2019-12-31T23:00:00Z' # To include the end of the previous working week...
requestTo = '2020-01-01T00:00:00Z'

In [7]:
usbRequestBase = 'https://api.usb.urbanobservatory.ac.uk/api/v2.0a/sensors/entity'

# Fetch a list of all the entities in the Urban Sciences Building
usbResources = []

requestPage = 1
response = None

filterMetrics = ['Room Occupied']
filterUnits = []

while response is None or len(response) > 1:
  print('Fetching page %u...' % requestPage)

  response = json.loads(
      urllib.request.urlopen(
        '%s?page=%u' % (usbRequestBase, requestPage)
      ).read().decode('utf-8')
  )['items']

  requestPage = requestPage + 1

  for entity in response:
    for feed in entity['feed']:
      for timeseries in feed['timeseries']:
        if 'latest' in timeseries and 'error' in timeseries['latest']:
          # No access to some of the personal data timeseries
          continue
        for link in timeseries['links']:
          if (link['rel'] == 'archives' and \
            'latest' in timeseries):

            if feed['metric'] in filterMetrics or \
               timeseries['unit']['name'] in filterUnits:
              usbResources.append({
                  'entity': entity['name'],
                  'roomNumber': entity['meta'].get('roomNumber', 'Unknown'),
                  'roomZone': entity['meta'].get('roomZone', 'None'),
                  'bmsId': feed['brokerage'][0]['sourceId'],
                  'metric': feed['metric'],
                  'units': timeseries['unit']['name'],
                  'timeseriesIri': link['href']
              })

print('Discovered %u entities.' % len(usbResources))

Fetching page 1...
Fetching page 2...
Fetching page 3...
Fetching page 4...
Fetching page 5...
Fetching page 6...
Fetching page 7...
Fetching page 8...
Fetching page 9...
Fetching page 10...
Fetching page 11...
Fetching page 12...
Fetching page 13...
Fetching page 14...
Fetching page 15...
Fetching page 16...
Fetching page 17...
Fetching page 18...
Fetching page 19...
Fetching page 20...
Fetching page 21...
Fetching page 22...
Fetching page 23...
Fetching page 24...
Fetching page 25...
Fetching page 26...
Fetching page 27...
Fetching page 28...
Fetching page 29...
Fetching page 30...
Fetching page 31...
Fetching page 32...
Fetching page 33...
Fetching page 34...
Fetching page 35...
Fetching page 36...
Fetching page 37...
Fetching page 38...
Fetching page 39...
Fetching page 40...
Fetching page 41...
Fetching page 42...
Fetching page 43...
Fetching page 44...
Fetching page 45...
Fetching page 46...
Fetching page 47...
Fetching page 48...
Fetching page 49...
Fetching page 50...
Fetching 

In [9]:
usbMetadata = pd.DataFrame.from_records(usbResources)

# Only the first 20 rows will be displayed below, but the rest will be stored to a file
usbMetadata.head(20)

,entity,roomNumber,roomZone,bmsId,metric,units,timeseriesIri
0,Urban Sciences Building: Floor 1,Unknown,None,125_WC's_1.032_Occ,Room Occupied,unknown,https://api.usb.urbanobservatory.ac.uk/api/v2/...
1,Urban Sciences Building: Floor 1 Accessible WC,Unknown,None,111_AccessibleWC_1.013_Occ,Room Occupied,unknown,https://api.usb.urbanobservatory.ac.uk/api/v2/...
2,Urban Sciences Building: Floor 1 Ambient Kitchen,Unknown,None,117_AmbientKitchen_1.018_Occ,Room Occupied,unknown,https://api.usb.urbanobservatory.ac.uk/api/v2/...
3,Urban Sciences Building: Floor 1 Collaboration,Unknown,None,105_Collaboration_1.005_Occ,Room Occupied,unknown,https://api.usb.urbanobservatory.ac.uk/api/v2/...
4,Urban Sciences Building: Floor 1 Collaboration,Unknown,None,109_Collaboration_1.010_Occ,Room Occupied,unknown,https://api.usb.urbanobservatory.ac.uk/api/v2/...
5,Urban Sciences Building: Floor 1 DIG Zone 1,Unknown,1,118_DIG_1.024_Z1_Occ,Room Occupied,unknown,https://api.usb.urbanobservatory.ac.uk/api/v2/...
6,Urban Sciences Building: Floor 1 DIG Zone 10,Unknown,10,118_DIG_1.024_Z10_Occ,Room Occupied,unknown,https://api.usb.urbanobservatory.ac.uk/api/v2/...
7,Urban Sciences Building: Floor 1 DIG Zone 11,Unknown,11,118_DIG_1.024_Z11_Occ,Room Occupied,unknown,https://api.usb.urbanobservatory.ac.uk/api/v2/...
8,Urban Sciences Building: Floor 1 DIG Zone 12,Unknown,12,118_DIG_1.024_Z12_Occ,Room Occupied,unknown,https://api.usb.urbanobservatory.ac.uk/api/v2/...
9,Urban Sciences Building: Floor 1 DIG Zone 13,Unknown,13,118_DIG_1.024_Z13_Occ,Room Occupied,unknown,https://api.usb.urbanobservatory.ac.uk/api/v2/...


In [10]:
usbMetadata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 227 entries, 0 to 226
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   entity         227 non-null    object
 1   roomNumber     227 non-null    object
 2   roomZone       227 non-null    object
 3   bmsId          227 non-null    object
 4   metric         227 non-null    object
 5   units          227 non-null    object
 6   timeseriesIri  227 non-null    object
dtypes: object(7)
memory usage: 12.5+ KB


In [7]:
def downloadTimeseries(columnName, sourceIri, startTime, endTime, daysPerRequest = 15):
  print('  [', end='')

  timeStart = datetime.datetime.strptime(startTime, "%Y-%m-%dT%H:%M:%SZ")
  timeEnd = datetime.datetime.strptime(endTime, "%Y-%m-%dT%H:%M:%SZ")

  tsRecords = []

  for windowStart in dateutil.rrule.rrule(
      dateutil.rrule.DAILY,
      interval=daysPerRequest,
      dtstart=timeStart,
      until=timeEnd
    ):

    windowEnd = windowStart + pd.Timedelta(days=daysPerRequest) - pd.Timedelta(seconds=1)
    if windowEnd > timeEnd:
      windowEnd = timeEnd

    windowResponse = json.loads(
        urllib.request.urlopen(
          '%s?startTime=%s&endTime=%s' % (sourceIri, windowStart.isoformat().replace('+00:00', 'Z'), windowEnd.isoformat().replace('+00:00', 'Z'))
        ).read().decode('utf-8')
    )['historic']['values']

    tsRecords.extend(windowResponse)
    print('.', end='')

  print(']')

  if len(tsRecords) < 1:
    return None
  
  dfTs = pd.DataFrame.from_records(tsRecords, exclude=['duration'])

  # Times in this API are in ISO8601
  dfTs['time'] = dfTs['time'].apply(lambda t: datetime.datetime.strptime(t, "%Y-%m-%dT%H:%M:%S.%fZ"))
  
  dfTs = dfTs.rename(columns={'value': columnName})
  dfTs.set_index('time', inplace=True, drop=True)

  return dfTs

In [8]:
# These timestamps are in ISO8601 format. You can Google this for more information.
# YYYY-MM-DDTHH:MM:SSZ - always in UTC, no British Summer Time
# Be aware that you probably need to request a little bit more than you want, because
# otherwise there could be gaps at the start and end where the value didn't change...
#requestFrom = '2019-03-01T00:00:00Z' # To include the end of the previous working week...
#requestTo = '2020-03-02T00:00:00Z'

In [9]:
dfRooms = None

print('Requesting USB room data...')

for idx, row in usbMetadata.iterrows():
  ts = row.to_dict()
  print('  %s' % ts['entity'])

  pdTs = downloadTimeseries(
    ts['bmsId'],
    ts['timeseriesIri'],
    requestFrom,
    requestTo
  )

  if pdTs is None:
    continue

  # The way the data is stored in the USB, is that unless the value changes, nothing
  # is stored. This does a forward fill, repeating the previous value until a new one
  # is identified.
  pdTs = pdTs.resample('%us' % resampleFrequency).ffill()

  if dfRooms is None:
    dfRooms = pdTs
  else:
    dfRooms = dfRooms.join(pdTs, how='outer')

dfRooms.head(20)

Requesting USB room data...
  Urban Sciences Building: Floor 1
  [.]
  Urban Sciences Building: Floor 1 Accessible WC
  [.]
  Urban Sciences Building: Floor 1 Ambient Kitchen
  [.]
  Urban Sciences Building: Floor 1 Collaboration
  [.]
  Urban Sciences Building: Floor 1 Collaboration
  [.]
  Urban Sciences Building: Floor 1 DIG Zone 1
  [.]
  Urban Sciences Building: Floor 1 DIG Zone 10
  [.]
  Urban Sciences Building: Floor 1 DIG Zone 11
  [.]
  Urban Sciences Building: Floor 1 DIG Zone 12
  [.]
  Urban Sciences Building: Floor 1 DIG Zone 13
  [.]
  Urban Sciences Building: Floor 1 DIG Zone 14
  [.]
  Urban Sciences Building: Floor 1 DIG Zone 15
  [.]
  Urban Sciences Building: Floor 1 DIG Zone 16
  [.]
  Urban Sciences Building: Floor 1 DIG Zone 17
  [.]
  Urban Sciences Building: Floor 1 DIG Zone 3
  [.]
  Urban Sciences Building: Floor 1 DIG Zone 4
  [.]
  Urban Sciences Building: Floor 1 DIG Zone 5
  [.]
  Urban Sciences Building: Floor 1 DIG Zone 6
  [.]
  Urban Sciences Building

  [.]
  Urban Sciences Building: Floor G Cafe Store
  [.]
  Urban Sciences Building: Floor G Computer Workshop
  [.]
  Urban Sciences Building: Floor G Cyber Physical
  [.]
  Urban Sciences Building: Floor G Decision Theatre
  [.]
  Urban Sciences Building: Floor G EPTCell
  [.]
  Urban Sciences Building: Floor G EPTCell
  [.]
  Urban Sciences Building: Floor G EPTControl
  [.]
  Urban Sciences Building: Floor G Female Showers
  [.]
  Urban Sciences Building: Floor G SUDS Laboratory
  [.]
  Urban Sciences Building: Floor G Servery
  [.]
  Urban Sciences Building: Floor G Shared Labs
  [.]
  Urban Sciences Building: Floor G Shared Work
  [.]
  Urban Sciences Building: Floor G Smart Grid
  [.]
  Urban Sciences Building: Floor G: Room G.009 Zone 1
  [.]
  Urban Sciences Building: Floor G: Room G.071 Zone 1
  [.]


,118_DIG_1.024_Z3_Occ,135_MeetingRoom_1.043_Occ,Zone1_BMSControl.RmHVAC.208_StudentRefresh_2.008_Occ
time,,,
2019-12-31 23:15:00,NaN,NaN,NaN
2019-12-31 23:30:00,1.0,1.0,NaN


In [10]:
# These timestamps are in ISO8601 format. You can Google this for more information.
# YYYY-MM-DDTHH:MM:SSZ - always in UTC, no British Summer Time
# Be aware that you probably need to request a little bit more than you want, because
# otherwise there could be gaps at the start and end where the value didn't change...
#requestFrom = '2019-01-07T00:00:00Z' # To include the end of the previous working week...
#requestTo = '2020-01-21T00:00:00Z'

In [11]:
# Write to an Excel file...

outputWriter = pd.ExcelWriter(
  'urban-sciences-building-data.xlsx',
  engine='xlsxwriter',
  datetime_format='mmm d yyyy hh:mm:ss',
  date_format='mmmm dd yyyy'
)

# Add sheets
usbMetadata.to_excel(outputWriter, sheet_name='Room metadata')
dfRooms.to_excel(outputWriter, sheet_name='Room occupancy')
#dfPower.to_excel(outputWriter, sheet_name='Power consumption')

outputWriter.save()